In [1]:
#MERGED DATA

import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, KFold
from sklearn import utils
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error # for calculating the cost function
from sklearn.ensemble import RandomForestRegressor # for building the model
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR


data = pd.read_csv('/Users/selinzobu/Desktop/TILES/111OMFITBASES111.csv', parse_dates = ['Timestamp'])
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
#fitbit["Timestamp"] = pd.to_datetime(fitbit["Timestamp"], utc=True)
#print(data.columns)

bases = pd.read_csv('/Users/selinzobu/Desktop/TILES/1BASESTRESSD.csv', parse_dates = ['Timestamp'])
bases = bases.loc[:, ~bases.columns.str.contains('^Unnamed')]

FOM = pd.read_csv('/Users/selinzobu/Desktop/TILES/1FITOMSTRESSD.csv', parse_dates = ['Timestamp'])
FOM = FOM.loc[:, ~FOM.columns.str.contains('^Unnamed')]

X = data.iloc[:,2:65]  #independent columns
X = X.drop('days', axis = 1)
y = data.iloc[:,-1]    #target column 

X_base = bases.drop(['participant_id', 'Timestamp', 'mstressd'], axis = 1)  #independent columns
y_base = bases['mstressd']    #target column 

X_FOM = FOM.drop(['participant_id', 'Timestamp', 'mstressd', 'days'], axis = 1)  #independent columns
y_FOM = FOM['mstressd']   #target column 

#for col in X.columns:
 #   X[col] = X[col].apply(pd.to_numeric, errors='coerce')

In [2]:
X2 = X[['AvgHeartRateS', 'RMSStdDev_msM', 'AvgBreathingRateM', 'AvgZAccel_gM', 'StdDevBreathingRateM', 'Fat Burn_caloriesOut', 'NumberSteps', 'iod_ID', 'iod_OD', 'ocb', 'pan_NegAffect', 'stai', 'rand_EmotionalWellbeing', 'swls', 'pss', 'mpfi_Flexibility_Defusion', 'pcq_Hope', 'pcq_Optimism', 'chss_ChallengeStressors', 'chss_HindranceStressors']]

X2_FOM = X_FOM[['Cardio_caloriesOut', 'AvgHeartRateS', 'RMSStdDev_msM', 'AvgBreathingRateM', 'AvgYAccel_gM', 'AvgZAccel_gM', 'StdDevBreathingRateM', 'StdDevBreathingRateS', 'StdDevGForceM', 'StdDevYAccel_gM']]

X2_base = X_base[['rand_EmotionalWellbeing', 'ocb', 'pcq_Hope', 'mpfi_Flexibility_Defusion', 'mpfi_Flexibility_SelfAsContext', 'iod_ID', 'mpfi_Flexibility_PresentMomentAwareness', 'swls', 'chss_ChallengeStressors', 'stai', 'pan_NegAffect']]

In [3]:
print(X2.shape, X2_FOM.shape, X2_base.shape, y.shape)

(2989, 20) (3126, 10) (7868, 11) (2989,)


In [4]:
X2_train, X2_test, y_train, y_test = train_test_split(
    X2, y, test_size=0.3, random_state=42)

X2b_train, X2b_test, yb_train, yb_test = train_test_split(
    X2_base, y_base, test_size=0.3, random_state=42)

X2FOM_train, X2FOM_test, yFOM_train, yFOM_test = train_test_split(
    X2_FOM, y_FOM, test_size=0.3, random_state=42)

In [5]:
print('X2_train:', X2_train.shape, 'X2_test:', X2_test.shape, 'y_train:', y_train.shape,'y_test:', y_test.shape)
#print(y_test.head())
print('X2b_train:', X2b_train.shape, 'X2b_test:', X2b_test.shape, 'yb_train:', yb_train.shape,'yb_test:', yb_test.shape)
#print(yb_test.head())
print('X2FOM_train:', X2FOM_train.shape, 'X2FOM_test:', X2FOM_test.shape, 'yFOM_train:', yFOM_train.shape,'yFOM_test:', yFOM_test.shape)
#print(yFOM_test.head())

X2_train: (2092, 20) X2_test: (897, 20) y_train: (2092,) y_test: (897,)
X2b_train: (5507, 11) X2b_test: (2361, 11) yb_train: (5507,) yb_test: (2361,)
X2FOM_train: (2188, 10) X2FOM_test: (938, 10) yFOM_train: (2188,) yFOM_test: (938,)


In [6]:
print(y_train.mean(), yb_train.mean(), yFOM_train.mean())


1.902963671128107 1.803068821499909 1.9079067641681902


In [7]:
#The baseline prediction for our case can be the historical stress score averages. 
#In other words, our baseline is the error we would get if we simply predicted the average
#stress score for all days.

# The baseline predictions are the historical averages
Mbaseline_preds = y_train.mean()
Bbaseline_preds = yb_train.mean()
FOMbaseline_preds = yFOM_train.mean()

# Baseline errors, and display average baseline error
Mbaseline_errors = abs(Mbaseline_preds - y_test)
Bbaseline_errors = abs(Bbaseline_preds - yb_test)
FOMbaseline_errors = abs(FOMbaseline_preds - yFOM_test)

print('Average baseline error merged: ', round(np.mean(Mbaseline_errors), 2))
print('Average baseline error baseline: ', round(np.mean(Bbaseline_errors), 2))
print('Average baseline error FOM: ', round(np.mean(FOMbaseline_errors), 2))

print(len(Mbaseline_errors), len(Bbaseline_errors), len(FOMbaseline_errors))
Mbaseline_errors = list(Mbaseline_errors)
Bbaseline_errors = list(Bbaseline_errors)
FOMbaseline_errors = list(FOMbaseline_errors)

Average baseline error merged:  0.75
Average baseline error baseline:  0.75
Average baseline error FOM:  0.73
897 2361 938


In [8]:
#TRAINING SCORE

#https://machinelearningmastery.com/how-to-transform-target-variables-for-regression-with-scikit-learn/

#X2_train, ny_train >> not scaled

from numpy import mean
from numpy import absolute
from numpy import loadtxt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error # for calculating the cost function


# prepare the model with input scaling
pipeline = Pipeline(steps=[('normalize', StandardScaler()), ('model', SVR(kernel = 'rbf'))])
print(pipeline)
# prepare the model with target scaling
model = TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())
print(model)
# evaluate model
cv = KFold(n_splits=10, shuffle=True, random_state=1)
#Used not scaled values as they will be scaled here
scorestrain = cross_val_score(model, X2_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scorestest = cross_val_score(model, X2_test, y_test, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
model.fit(X2_test, y_test)
y_pred = model.predict(X2_test)
# convert scores to positive
scorestrain = absolute(scorestrain)
rmse = float(format(np.sqrt(mean_squared_error(y_test, y_pred)), '.3f'))
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

# summarize the results TRAINING
s_mean = mean(scorestrain)
print('Mean MAE: %.3f' % (s_mean)) #Mean MAE: 0.571 > c1 gamma1 Mean MAE: 0.625
print("\nRMSE: ", rmse, '\nMAE: ', mae, '\nMSE: ', mse)
    

Pipeline(steps=[('normalize', StandardScaler()), ('model', SVR())])
TransformedTargetRegressor(regressor=Pipeline(steps=[('normalize',
                                                      StandardScaler()),
                                                     ('model', SVR())]),
                           transformer=StandardScaler())
Mean MAE: 0.571

RMSE:  0.662 
MAE:  0.45737230794465056 
MSE:  0.4386940002204391


In [9]:
# Importing required libraries
from seaborn import load_dataset, pairplot
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import numpy as np

In [10]:
#StandardScaler, the class we use to scale the data, takes in a 2D array; otherwise, 
#it returns an error.

print(y_train.shape, type(y_train))
ny_train = y_train.to_numpy()
ny_test = y_test.to_numpy()


ny_train = (ny_train).reshape(-1,1)
ny_test = (ny_test).reshape(-1,1)


print(ny_train.shape, type(ny_train))
print(ny_test.shape, type(ny_test))


StdS_Xt = StandardScaler()
StdS_yt = StandardScaler()
StdS_Xte = StandardScaler()
StdS_yte = StandardScaler()

XT = StdS_Xt.fit_transform(X2_train)
yT = StdS_yt.fit_transform(ny_train)
XTe = StdS_Xte.fit_transform(X2_test)
yTe = StdS_yte.fit_transform(ny_test)



(2092,) <class 'pandas.core.series.Series'>
(2092, 1) <class 'numpy.ndarray'>
(897, 1) <class 'numpy.ndarray'>


In [11]:
#SCALING TRAINING

#print("Scaled X_l:", XT)
#print("Scaled y_p:", yT)
#print(yT.ravel().shape,yT.shape)

In [11]:
# import the model
#XT = StdS_Xt.fit_transform(X2_train)
#yT = StdS_yt.fit_transform(ny_train)


from sklearn.svm import SVR
# create the model object
regressor = SVR(kernel = 'rbf')
# fit the model on the data
regressor.fit(X2_train, ny_train.ravel()) #not scaled
 
# Predicting the target values of the test set
y2_pred = regressor.predict(X2_test)

# Calculate the absolute errors
errors = abs(y2_pred - ny_test.ravel())
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')


#0.64 degrees TEST SCORE default
#Mean Absolute Error: 0.73 degrees. C =1, gamma =1
#Mean Absolute Error: 0.74 degrees. no scaling default
#Mean Absolute Error: 0.76 degrees. no scaling C = 1, gamma =1 

Mean Absolute Error: 0.74 degrees.


In [15]:
#CROSS VALIDATION - PARAMETER TUNING

In [ ]:
#XT = StdS_Xt.fit_transform(X2_train) x_train
#yT = StdS_yt.fit_transform(ny_train) y_train

In [12]:
#TRAINING SCORES

from sklearn.svm import SVR


def tune_rbfsvm(x_T, y_T):
        
    space = {'gamma': [0.001, 0.01, 0.1, 1, 10, 100], 'C': [0.001, 0.01, 0.1, 1, 10, 100]}
 
    grid_search = GridSearchCV(SVR(kernel = 'rbf'), param_grid=space,
            cv=KFold(n_splits=10, shuffle=True, random_state=1),
        scoring='neg_mean_absolute_error', n_jobs=-1)
    
    grid_result = grid_search.fit(x_T, y_T)
    best_params = grid_result.best_params_
    
#defining the model based on best parameters

    svm = SVR(kernel ='rbf', C = grid_search.best_params_['C'],
             gamma = grid_search.best_params_['gamma'])
    
# Perform K-Fold CV

    trainscores = cross_val_score(svm, x_T, y_T, cv=10, scoring='neg_mean_absolute_error')

    
    return 'bestparameters;', best_params, 'MAE:', trainscores.mean(), svm



#'MAE:', -0.7118372187928482
#('bestparameters;', {'C': 1, 'gamma': 1}, 'MAE:', -0.7118372187928482) scaled
#('bestparameters;', {'C': 100, 'gamma': 1}, 'MAE:', -0.7282753117633405) not scaled

In [ ]:
#MERGED FEATURES

print(tune_rbfsvm(XT, y_train.ravel())) # scaled
#('bestparameters;', {'C': 1, 'gamma': 0.1}, 'MAE:',-0.56688215 SVR(C=1, gamma=0.1))
print(tune_rbfsvm(XT, yT.ravel())) # scaled 2

print(tune_rbfsvm(X2_train, y_train.ravel())) # not scaled
#'bestparameters;', {'C': 0.001, 'gamma': 0.001}, 'MAE:', MAE:', -0.7278748413, SVR(C=0.001, gamma=0.001))

In [ ]:
#BASELINE NOT SCALED TRAINING

print(tune_rbfsvm(X2b_train, yb_train.ravel())) # not scaled
#('bestparameters;', {'C': 0.1, 'gamma': 0.1}, 'MAE:', -0.5572987216036964, SVR(C=0.1, gamma=0.1))

In [51]:
#FOM NOT SCALED TRAINING

print(tune_rbfsvm(X2FOM_train, yFOM_train.ravel())) # not scaled
#('bestparameters;', {'C': 0.1, 'gamma': 0.001}, 'MAE:', -0.726604782459138, SVR(C=0.1, gamma=0.001))

('bestparameters;', {'C': 0.1, 'gamma': 0.001}, 'MAE:', -0.726604782459138, SVR(C=0.1, gamma=0.001))


In [13]:
#Multimodal Errors

MSVM_errors = []
    
svm = SVR(kernel = 'rbf', C = 0.001, gamma = 0.001) # not scaled

#scores = cross_val_score(svm, X_test, y_test, cv=10, scoring='neg_mean_absolute_error')
    
svm.fit(X2_train, y_train.ravel())
y_pred = svm.predict(X2_test)
    
MSVM_errors = abs(y_pred - y_test)

mae = mean_absolute_error(y_test, y_pred)
    
print("\nMAE:", mae, len(MSVM_errors))



MAE: 0.7531155106939699 897


In [73]:
#TEST SCORES NOT SCALED - MULTIMODAL

from sklearn.metrics import mean_squared_error, mean_absolute_error # for calculating the cost function

def model_test(X_t, y_t, X_test, y_test):
    MSVM_errors = []
    
    svm = SVR(kernel = 'rbf', C = 0.001, gamma = 0.001) # not scaled

    #scores = cross_val_score(svm, X_test, y_test, cv=10, scoring='neg_mean_absolute_error')
    
    svm.fit(X_t, y_t)
    y_pred = svm.predict(X_test)

    rmse = float(format(np.sqrt(mean_squared_error(y_test, y_pred)), '.3f'))
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    return  "\nRMSE: ", rmse, '\nMAE: ', mae, '\nMSE: ', mse

In [14]:
#Baseline Errors

BSVM_errors = []

svm = SVR(kernel = 'rbf', C = 0.1, gamma = 0.1) # not scaled

#scores = cross_val_score(svm, X_test, y_test, cv=10, scoring='neg_mean_absolute_error')
    
svm.fit(X2b_train, yb_train)

y_pred = svm.predict(X2b_test)
    
mae = mean_absolute_error(yb_test, y_pred)

BSVM_errors = abs(y_pred - yb_test)
    
print("\nMAE:", mae, len(BSVM_errors))


MAE: 0.5574807085533002 2361


In [47]:
#TEST SCORES NOT SCALED - BASELINE

from sklearn.metrics import mean_squared_error, mean_absolute_error # for calculating the cost function

def model_test(X_t, y_t, X_test, y_test):
    
    svm = SVR(kernel = 'rbf', C = 0.1, gamma = 0.1) # not scaled

    #scores = cross_val_score(svm, X_test, y_test, cv=10, scoring='neg_mean_absolute_error')
    
    svm.fit(X_t, y_t)
    y_pred = svm.predict(X_test)
    
    
    rmse = float(format(np.sqrt(mean_squared_error(y_test, y_pred)), '.3f'))
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    return  "\nRMSE: ", rmse, '\nMAE: ', mae, '\nMSE: ', mse

In [15]:
#FOM Errors

FOMSVM_errors = []

svm = SVR(kernel = 'rbf', C = 0.1, gamma = 0.001) # not scaled
    
svm.fit(X2FOM_train, yFOM_train)

y_pred = svm.predict(X2FOM_test)
    
mae = mean_absolute_error(yFOM_test, y_pred)

FOMSVM_errors = abs(y_pred - yFOM_test)
    
print("\nMAE:", mae, len(FOMSVM_errors))


MAE: 0.7298642002822618 938


In [52]:
#TEST SCORES NOT SCALED - FOM

from sklearn.metrics import mean_squared_error, mean_absolute_error # for calculating the cost function

def model_test(X_t, y_t, X_test, y_test):
    
    svm = SVR(kernel = 'rbf', C = 0.1, gamma = 0.001) # not scaled

    #scores = cross_val_score(svm, X_test, y_test, cv=10, scoring='neg_mean_absolute_error')
    
    svm.fit(X_t, y_t)
    y_pred = svm.predict(X_test)
    
    
    rmse = float(format(np.sqrt(mean_squared_error(y_test, y_pred)), '.3f'))
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    return  "\nRMSE: ", rmse, '\nMAE: ', mae, '\nMSE: ', mse

In [74]:
#MERGED FEATURE SET NOT SCALED, TEST SCORES

print(model_test(X2_train, y_train.ravel(), X2_test, y_test.ravel()))#not scaled 0.753115

('\nRMSE: ', 0.906, '\nMAE: ', 0.7531155106939699, '\nMSE: ', 0.8205068525865323)


In [49]:
#BASELINE FEATURE SET NOT SCALED, TEST SCORES

print(model_test(X2b_train, yb_train.ravel(), X2b_test, yb_test.ravel()))#0.557480708

('\nRMSE: ', 0.782, '\nMAE: ', 0.5574807085533002, '\nMSE: ', 0.611032123243561)


In [53]:
#FOM FEATURE SET NOT SCALED, TEST SCORES

print(model_test(X2FOM_train, yFOM_train.ravel(), X2FOM_test, yFOM_test.ravel()))#0.557480708

('\nRMSE: ', 0.9, '\nMAE: ', 0.7298642002822618, '\nMSE: ', 0.8104910690074747)


In [29]:
#Scaled, inverse scaled

from sklearn.metrics import mean_squared_error, mean_absolute_error # for calculating the cost function

def model_test(X_t, y_t, X_test, y_test):
    
    svm = SVR(kernel = 'rbf', C = 1, gamma = 0.1) #scaled
   #svmNS = SVR(kernel = 'rbf', C = 0.001, gamma = 0.001) #not scaled

    #scores = cross_val_score(svm, X_test, y_test, cv=10, scoring='neg_mean_absolute_error')
    
    svm.fit(X_t, y_t)
    #y_pred = svm.predict(X_test)
    
    y_pred = svm.predict(StdS_Xte.transform(X_test)).reshape(-1,1)

    
    rmse = float(format(np.sqrt(mean_squared_error(y_test, y_pred)), '.3f'))
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    return  "\nRMSE: ", rmse, '\nMAE: ', mae, '\nMSE: ', mse

In [32]:
#MULTIMODAL -TEST RESULTS 

print(model_test(XT, y_train.ravel(), XTe, y_test.ravel())) #scaled nMAE: ', 0.76056422

('\nRMSE: ', 0.911, '\nMAE: ', 0.7605642258803795, '\nMSE: ', 0.8295389475435867)


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [18]:
'''
def tune_linsvm(x_T, y_T):
    
    space = {'C': [1, 100]}

    
    grid_search = GridSearchCV(estimator=SVR(kernel ='linear'), param_grid=space,
            cv=KFold(n_splits=5, shuffle=True, random_state=1),
        scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    
            
    grid_result = grid_search.fit(x_T, y_T)
    best_params = grid_result.best_params_
                                       
    svm = SVR(C = grid_search.best_params_['C'])
    
# Perform K-Fold CV
    trainscores = cross_val_score(svm, x_T, y_T, cv=10, scoring='neg_mean_absolute_error')

    
    return 'bestparameters;', best_params, 'MAE:', trainscores.mean()
#('bestparameters;', {'C': 100}, 'MAE:', -0.7985931475954857)  scaled
# '''

"\ndef tune_linsvm(x_T, y_T):\n    \n    space = {'C': [1, 100]}\n\n    \n    grid_search = GridSearchCV(estimator=SVR(kernel ='linear'), param_grid=space,\n            cv=KFold(n_splits=5, shuffle=True, random_state=1),\n        scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)\n    \n            \n    grid_result = grid_search.fit(x_T, y_T)\n    best_params = grid_result.best_params_\n                                       \n    svm = SVR(C = grid_search.best_params_['C'])\n    \n# Perform K-Fold CV\n    trainscores = cross_val_score(svm, x_T, y_T, cv=10, scoring='neg_mean_absolute_error')\n\n    \n    return 'bestparameters;', best_params, 'MAE:', trainscores.mean()\n#('bestparameters;', {'C': 100}, 'MAE:', -0.7985931475954857)  scaled\n# "

In [14]:
'''from sklearn.svm import SVR

def tune_polysvm(x_T, y_T):
    
    space = {'degree': [2, 5], 'C': [1000, 20000], 'coef0': [0, 1]}
                 
    
    grid_search = GridSearchCV(estimator=SVR(kernel ='poly'), param_grid=space,
    cv=KFold(n_splits=5, shuffle=True, random_state=1),
        scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    
            
    grid_result = grid_search.fit(x_T, y_T)
    best_params = grid_result.best_params_
                                       
    svm = SVR(C = grid_search.best_params_['C'])
    
# Perform K-Fold CV
    trainscores = cross_val_score(svm, x_T, y_T, cv=10, scoring='neg_mean_absolute_error')

    
    return 'bestparameters;', best_params, 'MAE:', trainscores.mean()
    '''

In [60]:
# SEPARATE ATTEMPT OF SVM
#https://optunity.readthedocs.io/en/latest/notebooks/notebooks/sklearn-svr.html

import math
import itertools
import optunity
import optunity.metrics
import sklearn.svm

# we explicitly generate the outer_cv decorator so we can use it twice
outer_cv = optunity.cross_validated(x=x_train, y=y_train, num_folds=3)

def compute_mse_standard(x_train, y_train, x_test, y_test):
    """Computes MSE of an SVR with RBF kernel and default hyperparameters.
    """
    model = sklearn.svm.SVR().fit(x_train, y_train)
    predictions = model.predict(x_test)
    return optunity.metrics.mse(y_test, predictions)

# wrap with outer cross-validation
compute_mse_standard = outer_cv(compute_mse_standard)

In [61]:
#compute_mse_standard() returns a three-fold cross-validation estimate of MSE for an SVR 
#with default hyperparameters.
compute_mse_standard()

0.7758651176070329

In [34]:
#We will create a function that returns MSE based on optimized hyperparameters, where we 
#choose a polynomial kernel in advance.

def compute_mse_poly_tuned(x_train, y_train, x_test, y_test):
    """Computes MSE of an SVR with RBF kernel and optimized hyperparameters."""

    # define objective function for tuning
    @optunity.cross_validated(x=x_train, y=y_train, num_iter=2, num_folds=5)
    def tune_cv(x_train, y_train, x_test, y_test, C, degree, coef0):
        model = sklearn.svm.SVR(C=C, degree=degree, coef0=coef0, kernel='poly').fit(x_train, y_train)
        predictions = model.predict(x_test)
        return optunity.metrics.mse(y_test, predictions)

    # optimize parameters
    optimal_pars, _, _ = optunity.minimize(tune_cv, 150, C=[1000, 20000], degree=[2, 5], coef0=[0, 1])
    print("optimal hyperparameters: " + str(optimal_pars))

    tuned_model = sklearn.svm.SVR(kernel='poly', **optimal_pars).fit(x_train, y_train)
    predictions = tuned_model.predict(x_test)
    return optunity.metrics.mse(y_test, predictions)

# wrap with outer cross-validation
compute_mse_poly_tuned = outer_cv(compute_mse_poly_tuned)

'''compute_mse_poly_tuned() returns a three-fold cross-validation estimate of MSE for an 
SVR with RBF kernel with tuned hyperparameters 1000<C<20000, 2<degree<5 and 0<coef0<1
with a budget of 150 function evaluations. Each tuple of hyperparameters is evaluated using
twice-iterated 5-fold cross-validation.'''

'compute_mse_poly_tuned() returns a three-fold cross-validation estimate of MSE for an \nSVR with RBF kernel with tuned hyperparameters 1000<C<20000, 2<degree<5 and 0<coef0<1\nwith a budget of 150 function evaluations. Each tuple of hyperparameters is evaluated using\ntwice-iterated 5-fold cross-validation.'

In [ ]:
compute_mse_poly_tuned()

In [20]:
#The polynomial kernel yields pretty good results when optimized, but maybe we can do even 
#better with an RBF kernel.


def compute_mse_rbf_tuned(x_train, y_train, x_test, y_test):
    """Computes MSE of an SVR with RBF kernel and optimized hyperparameters."""

    # define objective function for tuning
    @optunity.cross_validated(x=x_train, y=y_train, num_iter=2, num_folds=5)
    def tune_cv(x_train, y_train, x_test, y_test, C, gamma):
        model = sklearn.svm.SVR(C=C, gamma=gamma).fit(x_train, y_train)
        predictions = model.predict(x_test)
        return optunity.metrics.mse(y_test, predictions)

    # optimize parameters
    optimal_pars, _, _ = optunity.minimize(tune_cv, 150, C=[1, 100], gamma=[0, 50])
    print("optimal hyperparameters: " + str(optimal_pars))

    tuned_model = sklearn.svm.SVR(**optimal_pars).fit(x_train, y_train)
    predictions = tuned_model.predict(x_test)
    return optunity.metrics.mse(y_test, predictions)

# wrap with outer cross-validation
compute_mse_rbf_tuned = outer_cv(compute_mse_rbf_tuned)

In [21]:
'''compute_mse_rbf_tuned() returns a three-fold cross-validation estimate of MSE for an SVR
with RBF kernel with tuned hyperparameters 1<C<100 and 0<γ<5 with a budget of 150 function 
evaluations. Each tuple of hyperparameters is evaluated using twice-iterated 5-fold 
cross-validation.'''

compute_mse_rbf_tuned()

optimal hyperparameters: {'C': 42.464780148239306, 'gamma': 0.01732551601730581}
optimal hyperparameters: {'C': 84.19416015624999, 'gamma': 1.3927476169293715}
optimal hyperparameters: {'C': 4.656571032577705, 'gamma': 1.191587233088118}


0.8686365147179491

In [22]:
'''Optunity can optimize conditional search spaces, here the kernel family and depending on
which family the hyperparameterization (γ, degree, coef0, ...). We start by defining the 
search space (we will try the linear, polynomial and RBF kernel).'''

space = {'kernel': {'linear': {'C': [0, 100]},
                    'rbf': {'gamma': [0, 50], 'C': [1, 100]},
                    'poly': {'degree': [2, 5], 'C': [1000, 20000], 'coef0': [0, 1]}
                    }
         }

In [23]:
def compute_mse_all_tuned(x_train, y_train, x_test, y_test):
    """Computes MSE of an SVR with RBF kernel and optimized hyperparameters."""

    # define objective function for tuning
    @optunity.cross_validated(x=x_train, y=y_train, num_iter=2, num_folds=5)
    def tune_cv(x_train, y_train, x_test, y_test, kernel, C, gamma, degree, coef0):
        if kernel == 'linear':
            model = sklearn.svm.SVR(kernel=kernel, C=C)
        elif kernel == 'poly':
            model = sklearn.svm.SVR(kernel=kernel, C=C, degree=degree, coef0=coef0)
        elif kernel == 'rbf':
            model = sklearn.svm.SVR(kernel=kernel, C=C, gamma=gamma)
        else:
            raise ArgumentError("Unknown kernel function: %s" % kernel)
        model.fit(x_train, y_train)

        predictions = model.predict(x_test)
        return optunity.metrics.mse(y_test, predictions)

    # optimize parameters
    optimal_pars, _, _ = optunity.minimize_structured(tune_cv, num_evals=150, search_space=space)

    # remove hyperparameters with None value from optimal pars
    for k, v in optimal_pars.items():
        if v is None: del optimal_pars[k]
    print("optimal hyperparameters: " + str(optimal_pars))

    tuned_model = sklearn.svm.SVR(**optimal_pars).fit(x_train, y_train)
    predictions = tuned_model.predict(x_test)
    return optunity.metrics.mse(y_test, predictions)

# wrap with outer cross-validation
compute_mse_all_tuned = outer_cv(compute_mse_all_tuned)

In [ ]:
#And now the kernel family will be optimized along with its hyperparameterization.

compute_mse_all_tuned()

In [17]:
# SIGNIFICANCE OF DIFFERENCE

import scipy.stats as stats


print(len(FOMSVM_errors), len(FOMbaseline_errors))
print(len(BSVM_errors), len(Bbaseline_errors))
print(len(MSVM_errors), len(Mbaseline_errors))


938 938
2361 2361
897 897


In [18]:
#conduct the Wilcoxon-Signed Rank Test: SVM vs Baseline

Mdif = stats.wilcoxon(MSVM_errors, Mbaseline_errors)
FOMdif = stats.wilcoxon(FOMSVM_errors, FOMbaseline_errors)
Bdif = stats.wilcoxon(BSVM_errors, Bbaseline_errors)

print(Mdif)
print(FOMdif)
print(Bdif)

WilcoxonResult(statistic=179246.5, pvalue=0.004353793906067737)
WilcoxonResult(statistic=210977.0, pvalue=0.2666944649413523)
WilcoxonResult(statistic=758678.5, pvalue=5.1166597855801505e-82)


In [18]:
"{:.8f}".format(float("5.1166597855801505e-82"))

'0.00000000'

In [20]:
MRFERRORS = pd.read_csv('/Users/selinzobu/Desktop/TILES/1MRFERRORS.csv')
MRFERRORS = MRFERRORS.loc[:, ~MRFERRORS.columns.str.contains('^Unnamed')]

#MRFERRORS = MRFERRORS.values.tolist()

BRFERRORS = pd.read_csv('/Users/selinzobu/Desktop/TILES/1BRFERRORS.csv')
BRFERRORS = BRFERRORS.loc[:, ~BRFERRORS.columns.str.contains('^Unnamed')]

#BRFERRORS = BRFERRORS.values.tolist()

FOMRFERRORS = pd.read_csv('/Users/selinzobu/Desktop/TILES/1FOMRFERRORS.csv')
FOMRFERRORS = FOMRFERRORS.loc[:, ~FOMRFERRORS.columns.str.contains('^Unnamed')]

#FOMRFERRORS = FOMRFERRORS.values.tolist()
print(FOMRFERRORS.shape, FOMRFERRORS.squeeze().shape, type(FOMRFERRORS.squeeze()))

(938, 1) (938,) <class 'pandas.core.series.Series'>


In [21]:
print(type(MSVM_errors), type(MRFERRORS))

<class 'pandas.core.series.Series'> <class 'pandas.core.frame.DataFrame'>


In [22]:
#conduct the Wilcoxon-Signed Rank Test : RF vs SVM

Mdif = stats.wilcoxon( MRFERRORS.squeeze(), MSVM_errors)
FOMdif = stats.wilcoxon( FOMRFERRORS.squeeze(), FOMSVM_errors)
Bdif = stats.wilcoxon(BRFERRORS.squeeze(), BSVM_errors)

print(Mdif)
print(FOMdif)
print(Bdif)

WilcoxonResult(statistic=120789.0, pvalue=2.975169608234801e-25)
WilcoxonResult(statistic=173642.0, pvalue=2.034053896483224e-08)
WilcoxonResult(statistic=1166390.0, pvalue=6.162215274115193e-12)


In [23]:
'''WilcoxonResult(statistic=120789.0, pvalue=2.975169608234801e-25)
WilcoxonResult(statistic=173642.0, pvalue=2.034053896483224e-08)
WilcoxonResult(statistic=1166390.0, pvalue=6.162215274115193e-12)'''

'WilcoxonResult(statistic=120789.0, pvalue=2.975169608234801e-25)\nWilcoxonResult(statistic=173642.0, pvalue=2.034053896483224e-08)\nWilcoxonResult(statistic=1166390.0, pvalue=6.162215274115193e-12)'

In [22]:
#conduct the Wilcoxon-Signed Rank Test : MERGED VS SURVEY ONLY

MBRF = stats.wilcoxon( MRFERRORS.squeeze(), BRFERRORS.squeeze()) # RF: M vs B
MFOMRF = stats.wilcoxon(  MRFERRORS.squeeze(),  FOMMRFERRORS.squeeze()) #RF: M vs FOM


MBSVM = stats.wilcoxon(MSVM_errors, BSVM_errors) # SVM : M vs B
MFOMSVM = stats.wilcoxon(MSVM_errors, FOMSVM_errors) # SVM: M vs FOM


MBBASE = stats.wilcoxon(Mbaseline_errors, Bbaseline_errors) # BASE: M vs B
MFOMBASE = stats.wilcoxon(  Mbaseline_errors,  FOMbaseline_errors) #BASE: M vs FOM


print(MBRF)
print(MFOMRF)
print(MBSVM)
print(MFOMSVM)
print(MBBASE)
print(MFOMBASE)


ValueError: The samples x and y must have the same length.